In [2]:
from pymongo import MongoClient
import json

In [3]:
client = MongoClient("mongodb://localhost/")
db = client.admin

In [4]:
# Create database 
db = client["amazon"]
   
# Created or Switched to collection 
Collection = db["reviews"]

In [10]:
file_data = []
file = open('reviews_electronics.16.json', 'r')
for line in file:
    file_data.append(json.loads(line))
file.close()

In [11]:
db.reviews.insert_many(file_data)

## Use MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entires of "avg_scores" to screen.

In [17]:
import bson

In [20]:
mapFunction1 = bson.Code('''function() {
   emit(this.asin, this.overall);
}''')

reduceFunction1 = bson.Code('''function(keyasin, valuesoverall) {
   return Array.avg(valuesoverall);
}''')

db.reviews.map_reduce(
   mapFunction1,
   reduceFunction1,
   out = "map_reduce_example"
)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'amazon'), 'map_reduce_example')

In [22]:
list(db.map_reduce_example.find().limit(100))

[{'_id': 'B00E59WI7U', 'value': 5.0},
 {'_id': 'B00EWHMAI2', 'value': 4.571428571428571},
 {'_id': 'B00FP0S3SQ', 'value': 4.0},
 {'_id': 'B00GRZIF0O', 'value': 1.0},
 {'_id': 'B00EFLCEN6', 'value': 2.5},
 {'_id': 'B00KJJW36G', 'value': 4.454545454545454},
 {'_id': 'B00GGM6RB2', 'value': 3.8666666666666667},
 {'_id': 'B00G5TU4BK', 'value': 5.0},
 {'_id': 'B00FZKOEOS', 'value': 5.0},
 {'_id': 'B00ESLEAFI', 'value': 4.159420289855072},
 {'_id': 'B00EODJZ8C', 'value': 4.333333333333333},
 {'_id': 'B00ESMO37C', 'value': 4.0},
 {'_id': 'B00EU7YGG8', 'value': 1.0},
 {'_id': 'B00K0OBEE2', 'value': 3.623931623931624},
 {'_id': 'B00HUESTG6', 'value': 2.0},
 {'_id': 'B00FYYX7LG', 'value': 5.0},
 {'_id': 'B00K301TOI', 'value': 5.0},
 {'_id': 'B00GZMJ5SA', 'value': 5.0},
 {'_id': 'B00FZMIPDM', 'value': 4.5},
 {'_id': 'B00GGOXVKK', 'value': 3.5714285714285716},
 {'_id': 'B00GSLIW1Y', 'value': 5.0},
 {'_id': 'B00GOJWQ08', 'value': 4.0},
 {'_id': 'B00H7WGD9Q', 'value': 5.0},
 {'_id': 'B00GXFDCAQ', 'va

## Use MongoDB's aggregate function to build a new collection "weighted_avg_scores" that averages review scores by product ("asin"), weighted by the number of votes + 1 (the second number + 1). Print the first 100 entires of "weighted_avg_scores" to screen.

In [34]:
a=list(db.reviews.find({}, {"helpful":1, "_id":0}))

In [44]:
db.reviews2.insert_many(list(db.reviews.find()))

In [51]:
i=0
for res in db.reviews2.find():
    db.reviews2.update_one(res, {'$set':{'votes':a[i]["helpful"][1]+1}})
    i+=1

In [55]:
db.reviews2.aggregate([{
  "$group" : {
     "_id" : '$asin',
     "numerator": { "$sum": { "$multiply": [ "$overall", "$votes" ] } },
     "denominator": { "$sum": "$votes" }
  }
}, {
  "$project": {
    "average": { "$divide": [ "$numerator", "$denominator" ] }
  }
}, { 
    "$out": "weighted_avg_scores" }
])

In [57]:
list(db.weighted_avg_scores.find().limit(100))

[{'_id': 'B00IHUWW06', 'average': 3.0},
 {'_id': 'B00ELPIWIC', 'average': 4.14453125},
 {'_id': 'B00IFYUZ5I', 'average': 4.0},
 {'_id': 'B00HQPF7PU', 'average': 4.714285714285714},
 {'_id': 'B00KSV87W4', 'average': 5.0},
 {'_id': 'B00EPMOGPE', 'average': 2.7058823529411766},
 {'_id': 'B00ESCCO8M', 'average': 5.0},
 {'_id': 'B00EZ7NRQ8', 'average': 1.0},
 {'_id': 'B00GYOT8BS', 'average': 5.0},
 {'_id': 'B00HZ5JTS2', 'average': 5.0},
 {'_id': 'B00EPY0JD0', 'average': 4.25},
 {'_id': 'B00JA7MDFE', 'average': 1.0},
 {'_id': 'B00F19Q8ME', 'average': 4.371428571428571},
 {'_id': 'B00J2CCCJE', 'average': 4.2},
 {'_id': 'B00EA0VP7S', 'average': 4.333333333333333},
 {'_id': 'B00EYW28AU', 'average': 5.0},
 {'_id': 'B00KR216UE', 'average': 5.0},
 {'_id': 'B00GCQAT84', 'average': 5.0},
 {'_id': 'B00ESR0U2E', 'average': 4.666666666666667},
 {'_id': 'B00FN9CJFM', 'average': 4.0},
 {'_id': 'B00FY81YL2', 'average': 5.0},
 {'_id': 'B00FQT8T8U', 'average': 1.0},
 {'_id': 'B00GAGMWL8', 'average': 3.67567